In [14]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import re
np.set_printoptions(threshold=np.inf)
%matplotlib inline

edge_list_fused_afternorm = []
edge_list_ses1_afternorm = []
edge_list_ses2_afternorm = []
edge_list_ses1_normalized = []
edge_list_ses2_normalized = []
edge_list_property = []
length = 25920 - 25864 + 1

for i in range(1):
    p1 = r"./BNU3_dMRI/sub-00%s_ses-1_dwi_Desikan_space-MNI152NLin6_res-2x2x2_connectome.csv"%(int(i+25864))
    p2 = r"./BNU3_dMRI/sub-00%s_ses-1_dwi_Desikan_space-MNI152NLin6_res-2x2x2_connectome.csv"%(int(i+25864))

    edge_list_ses1 = []
    edge_list_ses2 = []
    
    with open(p1,encoding = 'utf-8') as f1:
        data1 = list(np.loadtxt(f1,str,delimiter = ","))
    for d in data1:
        d = d.split(' ')
        d=[round(float(x)) for x in d]
        edge_list_ses1.append(d)

    with open(p2,encoding = 'utf-8') as f2:
        data2 = list(np.loadtxt(f2,str,delimiter = ","))
    for d in data2:
        d = d.split(' ')
        d=[round(float(x)) for x in d]
        edge_list_ses2.append(d)
    
    G1 = nx.Graph()
    G1.add_weighted_edges_from(edge_list_ses1)
    A1 = nx.to_numpy_array(G1)
#     print(np.count_nonzero(A1))
    A1_degree = np.zeros(shape=(A1.shape))
    for k in range(A1.shape[0]):
        A1_degree[k][k] = list(G1.degree)[k][1]
    # print(-np.sqrt(A1_degree))
    A1_normalized = np.matmul(np.matmul((-np.sqrt(A1_degree)), A1), (-np.sqrt(A1_degree)))
    # A1_normalized = np.where(A1_normalized>0.5, 10*A1_normalized, 0)
    max_value = np.amax(A1_normalized)
    A1_normalized = A1_normalized/max_value
    A1_normalized = np.where(A1_normalized>0.4, A1_normalized, 0)
#     print(np.count_nonzero(A1_normalized))
    edge_list_ses1_normalized = list(zip(*np.nonzero(A1_normalized)))
#     print(edge_list_ses1_normalized)
    
    G2 = nx.Graph()
    G2.add_weighted_edges_from(edge_list_ses2)
    A2 = nx.to_numpy_array(G2)
#     print(np.count_nonzero(A2))
    A2_degree = np.zeros(shape=(A2.shape))
    for j in range(A2.shape[0]):
        A2_degree[j][j] = list(G2.degree)[j][1]
    # print(-np.sqrt(A1_degree))
    A2_normalized = np.matmul(np.matmul((-np.sqrt(A2_degree)), A2), (-np.sqrt(A2_degree)))
    # A1_normalized = np.where(A1_normalized>0.5, 10*A1_normalized, 0)
    max_value = np.amax(A2_normalized)
    A2_normalized = A2_normalized/max_value
    A2_normalized = np.where(A2_normalized>0.4, A2_normalized, 0)
    edge_list_ses2_normalized = list(zip(*np.nonzero(A2_normalized)))
#     print(np.count_nonzero(A2_normalized))
#     print(edge_list_ses2_normalized)
    
    for e1 in edge_list_ses1_normalized:
        edge_list_ses1_afternorm.append([i, e1[0], e1[1]])
    for e2 in edge_list_ses2_normalized:
        edge_list_ses2_afternorm.append([i, e2[0], e2[1]])
    
    
    # Simply fusing two sessions (can be updated)
    for e1 in edge_list_ses1_normalized:
        if e1 not in edge_list_ses2_normalized:
            edge_list_fused_afternorm.append([i, e1[0], e1[1]])
    for e2 in edge_list_ses2_normalized:
        edge_list_fused_afternorm.append([i, e2[0], e2[1]])

graph_idx = 0
f = csv.reader(open('./BNU1_fMRI/BNU1_phenotypic_data.csv','r'))
for j, row in enumerate(f):
    if j != 0 and j % 2 ==1:
        label = row[3]
        filter(str.isdigit, label)
        edge_list_property.append([graph_idx, int(label)-1, 70])
        graph_idx += 1
            
        
df = pd.DataFrame([iter for iter in edge_list_fused_afternorm], columns=['graph_id', 'source', 'target'])
df.to_csv('./BNU1_dMRI/dMRI_brain_graph_edges_fused.csv', index=False)
df = pd.DataFrame([iter for iter in edge_list_ses1_afternorm], columns=['graph_id', 'source', 'target'])
df.to_csv('./BNU1_dMRI/dMRI_brain_graph_edges_ses1.csv', index=False)
df = pd.DataFrame([iter for iter in edge_list_ses2_afternorm], columns=['graph_id', 'source', 'target'])
df.to_csv('./BNU1_dMRI/dMRI_brain_graph_edges_ses2.csv', index=False)
df = pd.DataFrame([iter for iter in edge_list_property], columns=['graph_id', 'label', 'num_nodes'])
df.to_csv('./BNU1_dMRI/dMRI_brain_graph_properties.csv', index=False)

print('Finished preprocessing dMRI graph!')

Finished preprocessing dMRI graph!
